In [1]:
#!/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
sys.path.append('/opt/ros/kinetic/lib/python2.7/dist-packages')

from ENV import ENV
from sensor_msgs.msg import LaserScan, Image, Imu
from gazebo_msgs.msg import ModelState, ModelStates
from gazebo_msgs.srv import SetModelState
from geometry_msgs.msg import Vector3Stamped, Twist
from collections import deque
from PIL import Image as iimage

import matplotlib
import matplotlib.pyplot as plt
import rospy
import scipy.misc
import numpy as np
import random
import time
import random
import pickle
import copy
import os

Using TensorFlow backend.


In [2]:
laser = None
velocity = None
vel = None
theta = None
pose = None
orientation = None
image = None
depth_img = None

In [3]:
def DepthCallBack(img):
    global depth_img
    depth_img = img.data

def callback_laser(msg):    
    global laser
    laser = msg    
    laser = laser.ranges 
    
def callback_camera(msg):
    global image
    image = np.frombuffer(msg.data, dtype=np.uint8)    
    image = np.reshape(image, [480,640,3]) 
    image = np.array(image)

def callback_laser(msg):    
    global laser
    laser = msg    
    laser = laser.ranges 

def crash_check(laser_data, velocity, theta, delta_depth):    
    laser_sensor = np.array(laser_data)
    laser_index = np.isinf(laser_sensor)
    laser_sensor[laser_index] = 30
    laser_sensor = np.array(laser_sensor[360:720])
    laser_sensor[laser_sensor < 0.1] = 30
    done = False
    vel_flag = False
    zone_1_flag = False    
    
    crash_reward = 0
    depth_reward = 0
    vel_reward = 0
    depth_value = (np.min(laser_sensor) - 0.5) / 2.0       
    
    # reward for zone 1
    if depth_value >= 0.4:         
        depth_reward = 0.4
        vel_flag = True        
        
    # reward for zone 2
    else:                                
        vel_factor = np.absolute(np.cos(velocity))
        _depth_reward = depth_value * vel_factor + delta_depth
        depth_reward = np.min([0.4, _depth_reward])           
        vel_flag = False        
        
    # reward for crash
    if np.min(laser_sensor) <= 1.2:
        done = True                
        vel_flag = False
    
    # reward for velocity
    else:
        if vel_flag:
            vel_reward = velocity * np.cos(theta)* 0.2                
            
        else:
            vel_reward = 0            
    # select reward
    if done:
        reward = -1.0
    else:
        reward = depth_reward + vel_reward  
        
    return done, reward, np.min(laser_sensor), depth_value

def show_figure(image):
    #show image using cv2    
#     image = cv2.resize(image, (256*2, 320*2), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('input image', image)    
    cv2.waitKey(1)
        
def state_callback(msg):
    global velocity, pose, orientation, vel, theta
    idx = msg.name.index("quadrotor")      
    
    pose = msg.pose[idx].position
    orientation = msg.pose[idx].orientation    
    vel = msg.twist[idx]
    
    velocity_x = vel.linear.x
    velocity_y = vel.linear.y    
    velocity = np.sqrt(velocity_x**2 + velocity_y**2)    
    theta = vel.angular.z

In [ ]:
if __name__ == '__main__':
    # Check the gazebo connection
    rospy.init_node('env', anonymous=True)
    
    # Class define
    env = ENV()   
    # Parameter setting for the simulation
    EPISODE = 500    
    global_step = 0 
    # Observe
    rospy.Subscriber('/camera/rgb/image_raw', Image, callback_camera,queue_size = 5)    
    rospy.Subscriber("/scan", LaserScan, callback_laser,queue_size = 5)
    rospy.Subscriber('gazebo/model_states', ModelStates, state_callback, queue_size= 5)      
    rospy.Subscriber('/camera/depth/image_raw', Image, DepthCallBack,queue_size = 5)
       
    # define command step
    rospy.sleep(2.)             
    rate = rospy.Rate(5)
    env.reset_sim(pose, orientation) 
    
    # define episode and image_steps
    e = 0
    count = 0
    global_step = 0
    episode_index = []
    while count < EPISODE and not rospy.is_shutdown():
        env.reset_sim(pose, orientation)                      
        
        step = 0                    
               
        done = False                    
        video_flag = False        
                
        image_buffer = []  
        image_steps = 0
        action_index = np.random.randint(7)
        side_index = np.random.randint(3)
        while not done and not rospy.is_shutdown():  
            # wait for service
            rospy.wait_for_message('/camera/rgb/image_raw', Image)
            rospy.wait_for_message('/camera/depth/image_raw', Image)
            rospy.wait_for_message('/gazebo/model_states', ModelStates)
            rospy.wait_for_message('/scan', LaserScan)           
                
            step = step + 1                       
            
            # Receive the action command from the Q-network and do the action                                    
            env.Control(action_index, side_index)                        
            
            # Observe: get_reward
            [done, reward, _depth, depth_value] = crash_check(laser, 0, 0, 0)
            
            if _depth <= 4.0 and _depth >= 1.2:
                video_flag = True
            
            show_figure(image)
            if not done and video_flag:                
                cv2.imwrite('./ttc_training_data/data_2/image_' + str(global_step) + '.jpg', image)
                global_step = global_step + 1
                                
            if done:                
                video_flag = False
                count = count + 1
                episode_index.append(global_step)
                
            rate.sleep()                           
        print("Current Episode:{}".format(count))

    np.save('./ttc_training_data/label/label_2.npy', episode_index)

[WARN] [1629207575.882753, 14841.090500]: Could not process inbound connection: topic types do not match: [geometry_msgs/PoseStamped] vs. [geometry_msgs/Pose]{'tcp_nodelay': '0', 'type': 'geometry_msgs/PoseStamped', 'topic': '/command/pose', 'callerid': '/gazebo', 'md5sum': 'd3812c3cbc69362b77dc0b19b345f8f5'}


Current Episode:1
Current Episode:2
Current Episode:3
Current Episode:4
Current Episode:5
Current Episode:6
Current Episode:7
Current Episode:8
Current Episode:9
Current Episode:10
Current Episode:11
Current Episode:12
Current Episode:13
Current Episode:14
Current Episode:15
Current Episode:16
Current Episode:17
Current Episode:18
Current Episode:19
Current Episode:20
Current Episode:21
Current Episode:22
Current Episode:23
Current Episode:24
Current Episode:25
Current Episode:26
Current Episode:27
Current Episode:28
Current Episode:29
Current Episode:30
Current Episode:31
Current Episode:32
Current Episode:33
Current Episode:34
Current Episode:35
Current Episode:36
Current Episode:37
Current Episode:38
Current Episode:39
Current Episode:40
Current Episode:41
Current Episode:42
Current Episode:43
Current Episode:44
Current Episode:45
Current Episode:46
Current Episode:47
Current Episode:48
Current Episode:49
Current Episode:50
Current Episode:51
Current Episode:52
Current Episode:53
Cu

Current Episode:417
Current Episode:418
Current Episode:419
Current Episode:420
Current Episode:421
Current Episode:422
Current Episode:423
Current Episode:424
Current Episode:425
Current Episode:426
Current Episode:427
Current Episode:428
Current Episode:429
Current Episode:430
Current Episode:431
Current Episode:432
Current Episode:433
Current Episode:434
Current Episode:435
Current Episode:436
Current Episode:437
Current Episode:438
Current Episode:439
Current Episode:440
Current Episode:441
Current Episode:442
Current Episode:443
Current Episode:444
Current Episode:445
Current Episode:446
Current Episode:447
Current Episode:448
Current Episode:449
Current Episode:450
Current Episode:451
Current Episode:452
Current Episode:453
Current Episode:454
Current Episode:455
Current Episode:456
Current Episode:457
Current Episode:458
Current Episode:459
Current Episode:460
Current Episode:461
Current Episode:462
Current Episode:463
Current Episode:464
Current Episode:465
Current Episode:466


Convert the image using the V2 version for the 'engineering_2F images'

In [ ]:
# remove the last label since it is not collided.
np.load('./ttc_training_data/label/label_1.npy')